### Wetterdaten für die gewählten Stationen einlesen und verarbeiten

- Pfade für die einzelnen Wetterdateien aus der von 04_daten_herunterladen erzeugten Datei datei_liste.txt einlesen
- Die 10 Dateien mit Wetterdaten einlesen
- Nicht relevante Spalten entfernen
- Datentyp für Spalten anpassen
- Daten auf den Zeitraum von 2014-2024 reduzieren
- Fehlerhafte Daten entfernen
- Daten der letzten 3 Jahre darstellen

In [7]:
# Datei datei_liste.txt einlesen
datei_liste <- readLines("datasets/orte/datei_liste.txt")

# Liste, um die Inhalte der Dateien zu speichern
wetterdaten <- list()

# Jede Datei aus der Liste einlesen
for (i in seq_along(datei_liste)) {
  # Der Pfad zur Datei
  dateipfad <- datei_liste[i]
  
  # Datei einlesen
  wetterdaten[[i]] <- read.table(dateipfad, 
    header = TRUE, 
    sep = ";", 
    fill = TRUE, 
    fileEncoding = "UTF-8")
  
  # Benenne das dataframe in der Liste nach der Stations ID
  name <- paste("station", wetterdaten[[i]]$STATIONS_ID[1], sep = "_")
  names(wetterdaten)[i] <- name
  # Nur relevante Spalten behalten
  wetterdaten[[i]] <- wetterdaten[[i]][, c("STATIONS_ID", "MESS_DATUM", "RSK", "SDK", "TMK")]

  # Datentypen anpassen
  wetterdaten[[i]]$MESS_DATUM <- as.Date(as.character(wetterdaten[[i]]$MESS_DATUM), format = "%Y%m%d")

  # Nur die letzten 10 Jahre behalten
  wetterdaten[[i]] <- tail(wetterdaten[[i]], 3652)

}

# Start- und Enddatum für den 3 Jahrezeitraum festlegen
start_datum <- as.Date("2021-01-01")
end_datum <- as.Date("2023-12-31")

In [8]:
# Entfernen von fehlerhaften Daten

for (i in seq_along(wetterdaten)) {
  wetterdaten[[i]] <- subset(wetterdaten[[i]], RSK <= 400.0 & RSK >= 0.0)
  wetterdaten[[i]] <- subset(wetterdaten[[i]], SDK <= 24 & SDK >= 0)
  wetterdaten[[i]] <- subset(wetterdaten[[i]], TMK <= 60 & TMK >= -60)
}

In [35]:
# Die bereinigten Datensätze für die weitere Verwendung speichern

saveRDS(wetterdaten, "datasets/rds/wetterdaten.rds")

In [9]:
# Verschiedene Informationen zu den Dateien anzeigen
# durch entfernen der Kommentarzeichen können die einzelnen Informationen ausgegeben werden

for (i in seq_along(wetterdaten)) {
  #print(names(wetterdaten)[i])
  #print(paste("Anzahl der Zeilen", names(wetterdaten[i]), ": ", nrow(wetterdaten[[i]])))
  #print(summary(wetterdaten[[i]]))
  #print(head(wetterdaten[[1]]))
  #print(nrow(wetterdaten[[1]]))
  #boxplot(wetterdaten[[i]]$TMK, main = names(wetterdaten)[i])
  #print(subset(wetterdaten[[i]], RSK > 400.0 | RSK < 0.0))
  #print(subset(wetterdaten[[i]], SDK > 24 | SDK < 0))
  #print(subset(wetterdaten[[i]], TMK > 60 | TMK < -60))
  #plot(wetterdaten[[i]]$MESS_DATUM, wetterdaten[[i]]$TMK, type = "l", col = "blue", xlab = "Datum", ylab = "Temperatur", main = names(wetterdaten)[i])
  #boxplot(wetterdaten[[i]]$TMK, main = names(wetterdaten)[i])
  #break
}

In [10]:
# Erste Auswertung der letzten 3 Jahre erzeugen und in Datei speichern - numerische Darstellung

for (i in seq_along(wetterdaten)) {
  daten_letzten_3_jahre <- subset(wetterdaten[[i]], MESS_DATUM >= start_datum & MESS_DATUM <= end_datum)
  sink(paste("outputs/", names(wetterdaten[i]), ".txt"))
  print(summary(daten_letzten_3_jahre))
  sink()
}

In [11]:
# Graphen für alle Orte der letzten 3 Jahre erstellen

for (i in seq_along(wetterdaten)) {
  daten_letzten_3_jahre <- subset(wetterdaten[[i]], MESS_DATUM >= start_datum & MESS_DATUM <= end_datum)
  
  #RSK
  png_name <- paste("plots/3years/", names(wetterdaten)[i], "_RSK.png", sep = "")
  png(png_name, width=800, height=400)
  plot(daten_letzten_3_jahre$MESS_DATUM, daten_letzten_3_jahre$RSK,
  type = "l",
  col = "blue",
  xlab = "Jahr",
  ylab = "RSK - tägliche Niederschlagshöhe in mm",
  main = names(wetterdaten)[i])
  dev.off()
  
  #SDK
  png_name <- paste("plots/3years/", names(wetterdaten)[i], "_SDK.png", sep = "")
  png(png_name, width=800, height=400)
  plot(daten_letzten_3_jahre$MESS_DATUM, daten_letzten_3_jahre$SDK,
  type = "p",
  col = "orange",
  xlab = "Jahr",
  ylab = "SDK - tägliche Sonnenscheindauer in h ",
  main = names(wetterdaten)[i])
  dev.off()
  
  #TMK
  png_name <- paste("plots/3years/", names(wetterdaten)[i], "_TMK.png", sep = "")
  png(png_name, width=800, height=400)
  plot(daten_letzten_3_jahre$MESS_DATUM, daten_letzten_3_jahre$TMK,
  type = "l",
  col = "green",
  xlab = "Jahr",
  ylab = "TMK - Tagesmittel Lufttemperatur in °C in 2 m Höhe",
  main = names(wetterdaten)[i])
  dev.off()
  
}

In [12]:
# Überlagerte Graphen aller Orte der letzten 3 Jahre
daten_letzten_3_jahre <- subset(wetterdaten[[1]], MESS_DATUM >= start_datum & MESS_DATUM <= end_datum)
colors = list("green", "blue", "orange", "red", "purple", "black", "yellow", "pink", "brown", "grey")

#TMK
png_name <- paste("plots/3years/", "alle", "_TMK.png", sep = "")
png(png_name, width=800, height=400)
plot(daten_letzten_3_jahre$MESS_DATUM, 
    daten_letzten_3_jahre$TMK, 
    type = "n", col = "green", 
    xlab = "Jahr", 
    ylab = "TMK - Tagesmittel Lufttemperatur in °C in 2 m Höhe", 
    main = "Überlagerte TMK-Graphen der letzten 3 Jahre"
    )

for (i in seq_along(wetterdaten)) {
  daten_letzten_3_jahre <- subset(wetterdaten[[i]], MESS_DATUM >= start_datum & MESS_DATUM <= end_datum)
  lines(daten_letzten_3_jahre$MESS_DATUM, daten_letzten_3_jahre$TMK, type = "l", col = colors[[i]])
}
dev.off()

#RSK
png_name <- paste("plots/3years/", "alle", "_RSK.png", sep = "")
png(png_name, width=800, height=400)
plot(daten_letzten_3_jahre$MESS_DATUM,
  daten_letzten_3_jahre$RSK,
  type = "n", 
  col = "blue", 
  xlab = "Jahr", 
  ylab = "RSK - tägliche Niederschlagshöhe in mm", 
  main = "Überlagerte RSK-Graphen der letzten 3 Jahre"
  )
  
for (i in seq_along(wetterdaten)) {
  daten_letzten_3_jahre <- subset(wetterdaten[[i]], MESS_DATUM >= start_datum & MESS_DATUM <= end_datum)
  lines(daten_letzten_3_jahre$MESS_DATUM, daten_letzten_3_jahre$RSK, type = "l", col = colors[[i]])
}
dev.off()

#SDK
png_name <- paste("plots/3years/", "alle", "_SDK.png", sep = "")
png(png_name, width=800, height=400)
plot(daten_letzten_3_jahre$MESS_DATUM,
  daten_letzten_3_jahre$SDK,
  type = "n", 
  col = "orange", 
  xlab = "Jahr", 
  ylab = "SDK - tägliche Sonnenscheindauer in h", 
  main = "Überlagerte SDK-Graphen der letzten 3 Jahre"
  )

for (i in seq_along(wetterdaten)) {
  daten_letzten_3_jahre <- subset(wetterdaten[[i]], MESS_DATUM >= start_datum & MESS_DATUM <= end_datum)
  lines(daten_letzten_3_jahre$MESS_DATUM, daten_letzten_3_jahre$SDK, type = "p", col = colors[[i]])
}
dev.off()

png 
  2

png 
  2

png 
  2

## Variabilitätsscore

In [32]:
# Matrix für Varianz und Standardabweichung (6 Spalten, n Zeilen)
n <- length(wetterdaten)
ergebnis_matrix <- matrix(numeric(n * 6), nrow = n, ncol = 6)

# Benenne die Spalten der Matrix
colnames(ergebnis_matrix) <- c("Varianz_RSK", "Stda_RSK", 
                               "Varianz_SDK", "Stda_SDK", 
                               "Varianz_TMK", "Stda_TMK")

for (i in seq_along(wetterdaten)) {
  ergebnis_matrix[i, 1] <- var(wetterdaten[[i]]$RSK, na.rm = TRUE)
  ergebnis_matrix[i, 2] <- sqrt(ergebnis_matrix[i, 1])
  
  ergebnis_matrix[i, 3] <- var(wetterdaten[[i]]$SDK, na.rm = TRUE)
  ergebnis_matrix[i, 4] <- sqrt(ergebnis_matrix[i, 3])
  
  ergebnis_matrix[i, 5] <- var(wetterdaten[[i]]$TMK, na.rm = TRUE)
  ergebnis_matrix[i, 6] <- sqrt(ergebnis_matrix[i, 5])
}
print(ergebnis_matrix)


      Varianz_RSK Stda_RSK Varianz_SDK Stda_SDK Varianz_TMK Stda_TMK
 [1,]    39.06270 6.250016    21.71427 4.659858    54.41073 7.376363
 [2,]    19.18963 4.380597    19.73699 4.442633    37.61751 6.133311
 [3,]    79.90281 8.938837    20.05801 4.478617    50.34378 7.095335
 [4,]    50.11535 7.079220    21.28061 4.613091    55.95097 7.480038
 [5,]    31.45721 5.608673    22.05088 4.695837    54.35087 7.372305
 [6,]    16.42201 4.052408    20.77976 4.558483    33.75741 5.810113
 [7,]    77.30649 8.792411    16.39719 4.049344    56.84773 7.539743
 [8,]    16.99618 4.122642    21.23359 4.607992    56.47594 7.515048
 [9,]    14.38721 3.793048    23.10893 4.807175    42.68266 6.533197
[10,]    14.38272 3.792455    19.78939 4.448527    48.66625 6.976120


In [34]:
ergebnis_df <- as.data.frame(ergebnis_matrix)
print(ergebnis_df)

   Varianz_RSK Stda_RSK Varianz_SDK Stda_SDK Varianz_TMK Stda_TMK
1     39.06270 6.250016    21.71427 4.659858    54.41073 7.376363
2     19.18963 4.380597    19.73699 4.442633    37.61751 6.133311
3     79.90281 8.938837    20.05801 4.478617    50.34378 7.095335
4     50.11535 7.079220    21.28061 4.613091    55.95097 7.480038
5     31.45721 5.608673    22.05088 4.695837    54.35087 7.372305
6     16.42201 4.052408    20.77976 4.558483    33.75741 5.810113
7     77.30649 8.792411    16.39719 4.049344    56.84773 7.539743
8     16.99618 4.122642    21.23359 4.607992    56.47594 7.515048
9     14.38721 3.793048    23.10893 4.807175    42.68266 6.533197
10    14.38272 3.792455    19.78939 4.448527    48.66625 6.976120


In [14]:
varianz <- var(wetterdaten[[1]]$RSK)
stda <- sd(wetterdaten[[1]]$RSK)
print(paste("Varianz: ", varianz))
print(paste("Standardabweichung: ", stda))

[1] "Varianz:  39.0626988926645"
[1] "Standardabweichung:  6.25001591139291"


In [19]:
varianz_rsk

[[1]]
[1] 39.0627

[[2]]
[1] 19.18963

[[3]]
[1] 79.90281

[[4]]
[1] 50.11535

[[5]]
[1] 31.45721

[[6]]
[1] 16.42201

[[7]]
[1] 77.30649

[[8]]
[1] 16.99618

[[9]]
[1] 14.38721

[[10]]
[1] 14.38272

In [29]:
var_stda <- data.frame("Ort", "v_RSK", "v_SDK", "v_TMK", "s_RSK", "s_SDK", "s_TMK")
var_stda <- data.frame("Ort" = names(wetterdaten), v_RSK = c(varianz_rsk))
var_stda

Ort,v_RSK.39.0626988926645,v_RSK.19.1896312561028,v_RSK.79.9028117423977,v_RSK.50.1153530143582,v_RSK.31.4572147607298,v_RSK.16.4220126860692,v_RSK.77.3064867543138,v_RSK.16.9961753730392,v_RSK.14.3872136239839,v_RSK.14.3827164295066
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
station_856,39.0627,19.18963,79.90281,50.11535,31.45721,16.42201,77.30649,16.99618,14.38721,14.38272
station_891,39.0627,19.18963,79.90281,50.11535,31.45721,16.42201,77.30649,16.99618,14.38721,14.38272
station_1346,39.0627,19.18963,79.90281,50.11535,31.45721,16.42201,77.30649,16.99618,14.38721,14.38272
station_2290,39.0627,19.18963,79.90281,50.11535,31.45721,16.42201,77.30649,16.99618,14.38721,14.38272
station_2712,39.0627,19.18963,79.90281,50.11535,31.45721,16.42201,77.30649,16.99618,14.38721,14.38272
station_3631,39.0627,19.18963,79.90281,50.11535,31.45721,16.42201,77.30649,16.99618,14.38721,14.38272
station_3730,39.0627,19.18963,79.90281,50.11535,31.45721,16.42201,77.30649,16.99618,14.38721,14.38272
station_3987,39.0627,19.18963,79.90281,50.11535,31.45721,16.42201,77.30649,16.99618,14.38721,14.38272
station_4271,39.0627,19.18963,79.90281,50.11535,31.45721,16.42201,77.30649,16.99618,14.38721,14.38272


In [28]:
varianz_rsk
remove(var_stda)

[[1]]
[1] 39.0627

[[2]]
[1] 19.18963

[[3]]
[1] 79.90281

[[4]]
[1] 50.11535

[[5]]
[1] 31.45721

[[6]]
[1] 16.42201

[[7]]
[1] 77.30649

[[8]]
[1] 16.99618

[[9]]
[1] 14.38721

[[10]]
[1] 14.38272

Warning message in remove(var_stda):
“Objekt 'var_stda' nicht gefunden”
